# Generate Binary Threshold

In [ ]:
import matplotlib.pylab as plt
import cv2
import numpy as np

global first_frame
global filter_flag
first_frame = 1

def binary_threshold(img, low, high):
    output = np.zeros_like(img[:,:,0])
    mask = (img[:,:,0] >= low[0]) & (img[:,:,0] <= high[0]) \
        & (img[:,:,1] >= low[1]) & (img[:,:,1] <= high[1]) \
        & (img[:,:,2] >= low[2]) & (img[:,:,2] <= high[2])
            
    output[mask] = 1
    return output

def get_binary_image(image):
    global first_frame
    global filter_flag
    
    ### HLS color space
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    L = hls[:,:,1]
    L_max, L_mean = np.max(L), np.mean(L)
    print(L_mean)
    S = hls[:,:,2]
    S_max, S_mean = np.max(S), np.mean(S)

    #Changing filter cooficients based on the average lightning
    if (first_frame):
        if ((L_mean < 123) | (L_mean > 126)):
            filter_flag = 0
        else:
            filter_flag = 1
            
    if (not filter_flag):
        #yellow cooficients
        yellow_L_thr = 80
        ye_L_mean_coof = 1.25
        S_max_coof = 0.25
        S_mean_coof = 1.75
        #white cooficients
        white_L_thr = 160
        L_max_coof = 0.8
        wh_L_mean_coof = 1.25
    else:
        #yellow cooficients
        yellow_L_thr = 0
        ye_L_mean_coof = 1
        S_max_coof = 0.1
        S_mean_coof = 1.7
        #white cooficients
        white_L_thr = 0
        L_max_coof = 0.7
        wh_L_mean_coof = 1.3
        
        
    # YELLOW
    L_adapt_yellow = max(yellow_L_thr, int(L_mean * ye_L_mean_coof))
    S_adapt_yellow = max(int(S_max * S_max_coof), int(S_mean * S_mean_coof))
    S_adapt_yellow = S_adapt_yellow if (not filter_flag) else 25
    hls_low_yellow = np.array((15, L_adapt_yellow, S_adapt_yellow))
    hls_high_yellow = np.array((30, 255, 255))

    hls_yellow = binary_threshold(hls, hls_low_yellow, hls_high_yellow)

    # WHITE
    L_adapt_white =  max(white_L_thr, int(L_max *L_max_coof),int(L_mean * wh_L_mean_coof))
    hls_low_white = np.array((0, L_adapt_white,  0))
    hls_high_white = np.array((255, 255, 255))

    hls_white = binary_threshold(hls, hls_low_white, hls_high_white)

    
    hls_binary = hls_yellow | hls_white
    
    #show
#     plt.imshow(hls_binary)
    
    return  hls_binary 


# PerspectiveTransform

In [ ]:
def perspectiveTrnsform(srcp ,dstp):
   M = cv2.getPerspectiveTransform(srcp ,dstp)
   Minv = cv2.getPerspectiveTransform(dstp ,srcp)
   return M ,Minv

def warpPerspective(img,imgsize,M):
    return cv2.warpPerspective(img,M,imgsize,cv2.INTER_LINEAR)

global src_first_pt, src_last_pt, dst_first_pt, dst_last_pt
def warp(image):
    global filter_flag
    global src_first_pt, src_last_pt, dst_first_pt, dst_last_pt

    if (not filter_flag):
        srcp = np.float32(
        [[685, 450],
          [1090, 710],
          [220, 710],
          [595, 450]])
        dstp = np.float32(
        [[900, 0],
          [900, 710],
          [250, 710],
          [250, 0]])
        
        src_first_pt = 220
        src_last_pt = 1090
        dst_first_pt = 250
        dst_last_pt = 900
    else:
        srcp = np.array([
            [200,713],
            [630,460],
            [750,460],
            [1200,713]
        ]).astype(np.float32)
        dstp = np.array([
            [300,713],
            [300,0],
            [1100,0],
            [1100,713]
        ]).astype(np.float32)
        
        src_first_pt = 200
        src_last_pt = 1200
        dst_first_pt = 300
        dst_last_pt = 1100
    
    height = image.shape[0]
    width = image.shape[1]
    imgsize2 =(width,height)
    M,Minv = perspectiveTrnsform(srcp ,dstp)
    wraped_img = warpPerspective(image.astype(np.float32),imgsize2,M)
    print("Height:",height,", Width:",width)
    
    #show
#     plt.imshow(wraped_img)
    
    return wraped_img, Minv


# Histogram

In [ ]:
def get_histogram(binary_warped):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    #show
#     plt.plot(histogram)
    
    return histogram

# sliding window

# Lane Curvature

# Draw Lane Lines

# Proccess Image

# Create Output video